In [4]:
import pandas as pd
from sklearn.linear_model import Ridge

# Load your data
df = pd.read_csv("nfl_ra_epa_over_replacement_train.csv")

# Columns to keep
features = [
    'pass_share', 'completion_percentage', 'yards_per_completion',
    'tds_per_completion', 'rush_share', 'yards_per_rush', 'tds_per_rush',
    'target_share', 'catch_percentage', 'yards_per_reception', 'tds_per_reception'
]
target = 'ra_epa_over_replacement'

# Drop irrelevant columns
df = df.drop(columns=['gsis_id', 'player_display_name', 'total_games'], errors='ignore')

# Prepare output container
results = {}

# Loop through position groups
for position, group in df.groupby('position_group'):
    X = group[features]
    y = group[target]

    # Drop rows with NaNs
    X = X.dropna()
    y = y.loc[X.index]

    # Skip if no rows remain
    if len(X) == 0:
        continue

    # Fit model
    model = Ridge(alpha=1.0).fit(X, y)
    
    # Store coefficients
    coef = pd.Series(model.coef_, index=features)
    coef['intercept'] = model.intercept_
    
    results[position] = coef


# Create output DataFrame
coef_df = pd.DataFrame(results)

# Save to CSV
coef_df.to_csv("position_group_coefficients.csv")

# Optional preview
print(coef_df)


                             QB            RB            TE            WR
pass_share            -0.014141 -1.367447e-02  2.019483e-03  9.740528e-04
completion_percentage  0.005456 -1.240675e-06 -3.550656e-06  1.679940e-05
yards_per_completion   0.052490 -2.186947e-06  6.757011e-05  4.444844e-05
tds_per_completion     0.001522  4.119823e-08  1.045742e-08  5.792176e-09
rush_share            -0.004368  3.467483e-03  7.484835e-04  1.410123e-04
yards_per_rush        -0.033409  4.116535e-02  5.494632e-05 -3.592550e-04
tds_per_rush          -0.000348  5.496341e-04  2.913515e-06 -1.278533e-05
target_share          -0.000007 -5.846310e-03 -1.208152e-03 -2.145469e-03
catch_percentage       0.002766  1.325593e-03 -2.234637e-03 -6.028878e-03
yards_per_reception   -0.028546  3.105346e-02  2.072790e-02  1.752939e-02
tds_per_reception     -0.000029 -1.671175e-05  6.402481e-04  1.481320e-03
intercept             -0.107169 -4.135518e-01 -2.134666e-01 -2.063631e-01


/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/riley.gisseman/Downloads/cfb_nfl_dev_env/.venv/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: 